In [ ]:
#need pip install spotipy to run this
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import base64
from requests import post, get
from config import cid, secret

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
results = sp.search(q='artist:' + 'Drake', type='artist')
print(results)

In [ ]:
auth_string = cid + ":" + secret
auth_bytes = auth_string.encode('utf-8')
auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')
url = "https://accounts.spotify.com/api/token"
headers = {
    'Authorization': 'Basic ' + auth_base64,
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {"grant_type": "client_credentials"}
result = post(url, headers=headers, data=data)
json_result = json.loads(result.content)
token = json_result['access_token']
print(token)

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX7Jl5KP2eZaS?si=fa88101bc2b04d96"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [ ]:
sp.playlist_tracks(playlist_URI)['items'][0]['track']['duration_ms']

In [ ]:
genres = ''
song_name = []
artist_name = []
artist_pop = []
genre_list = []
album = []
track_pop = []
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    #print(track_uri)
    #track = sp.track(track_uri)
    #print(track)
    #break
    song_genre = sp.track(track_uri)
    #print(song_genre['genre'])
    
    #Track name
    song_name.append(track["track"]["name"])
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    #print(f"Uri:{artist_uri}")
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name.append(track["track"]["artists"][0]["name"])
    artist_pop.append(artist_info["popularity"])
    artist_genres = artist_info["genres"]
    for genre in artist_genres:
        if genre == artist_genres[-1]:
            genres = genres + genre + ' '
        else: 
            genres = genres + genre + ', '
    genre_list.append(genres)
    genres = ''
    #Album
    album.append(track["track"]["album"]["name"])
    
    song_duration.append(track['track']['duration_ms'])
    
    #Popularity of the track
    track_pop.append(track["track"]["popularity"])
top_fifty_df = pd.DataFrame({"Song Name": song_name, 
                            "Artist Name": artist_name,
                            "Artist Popularity": artist_pop,
                            "Song Genre": genre_list,
                            "Album": album,
                            "Song Popularity": track_pop})
top_fifty_df

In [ ]:
artist_name = 'Drake'
url = 'https://api.spotify.com/v1/search'
headers = {"Authorization": "Bearer " + token}
query = f"?q={artist_name}&type=artist&limit=1"
query_url = url + query
result = get(query_url, headers=headers)
json_result = json.loads(result.content)
json_result['artists']

In [ ]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,10000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
